In [2]:
#import fwiVis.fwiVis as fv
import s3fs
s3 = s3fs.S3FileSystem(anon=False)
from math import cos, asin, sqrt
import re

import numpy as np
import geopandas as gpd
import pandas as pd
from matplotlib import pyplot as plt
import os
import rioxarray as rio
import xarray as xr
import rasterio
import glob
from shapely.errors import ShapelyDeprecationWarning
from shapely.geometry import Point
import warnings
import folium
import datetime
import time
from folium import plugins
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
import contextily as cx
from shapely.geometry import box
import sys
from datetime import datetime, timedelta
from itertools import chain

from datetime import date


sys.path.insert(0, '/projects/old_shared/fire_weather_vis/base-fwi-vis/')
import fwiVis.fwiVis as fv

In [280]:
### Functions that take largefire perimeters that are different overlapping fire and merge them into one. 
def prep_fire_files(path, crs = "3571"):
    '''
    Read in cvs of previously output GeoDataFrame as a GeoDataFrame with valid geometry column. 
    '''
    fires = pd.read_csv(path)
    fires = fires.rename(columns={"geometry":"csv_geometry"})
    fires.t = fires.t.astype("str")
    fires.fireID  = fires.fireID.astype("str")
    #fires['csv_geometry'] =fires['csv_geometry'].apply(wkt.loads)
    fires_geom = gpd.read_file(path, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
    fires_geom.t = fires_geom.t.astype("str")
    fires_geom.fireID  = fires_geom.fireID.astype("str")
    fires = fires_geom[["fireID", "t", "geometry"]].merge(fires, on=["fireID", "t"], how = "left")
    fires = fires.set_crs(crs)
    return (fires)

def remerge_largefire(fires):
    '''
    If two final feds perimeters intersect spatialy, check if one ended before the other began. If yes, give it the ID of the earlier perimeter. 
    Note: Could optionally be spitting out time differences, or sorting by them. For Quebec, was a max of 22 days, min of 1 day. Theoretically, not sure why the 1 day perimeter wasn't merged by feds. 
    '''
    # Get an id, first/ last t per ID, and a geometry
    first_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.min().reset_index()
    last_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.max().reset_index()
    plot_last = fires.merge(last_perims, on = ["fireID", "t"], how = 'right')
    plot_first = fires.merge(first_perims, on = ["fireID", "t"], how = 'right')
    plot_last = plot_last[["fireID", "t", "geometry"]]
    plot_first = plot_first[["fireID", "t", "geometry"]]
                    
    # Check what perimeters spatially intersect into anouther one through time
    last_in_last = plot_last.sjoin(plot_last, how = "left", predicate = "intersects")
    lil = last_in_last.groupby(["fireID_left",]).t_right.min().reset_index()
    id_with_max_time = lil.merge(last_in_last[["t_right", "t_left", "geometry", "fireID_right", "fireID_left"]], on = ["t_right", "fireID_left"], how = "right")

    id_with_max_time = id_with_max_time.rename(columns={"fireID_right": "mergeID", 
                                     "fireID_left" :"fireID",
                                     "t_right":"mergeID_t",
                                     "t_left":"fireID_end_t"
    })

    id_with_max_time_check = id_with_max_time[id_with_max_time.fireID_end_t < id_with_max_time.mergeID_t]

    ### FireID the earlier perimeter that later perimeters are merged into. "mergeID" describes the merge-ey
    fireID_with_merge = id_with_max_time_check.groupby(["fireID"]).mergeID.unique().reset_index() 

    mergeID_with_fireID  = id_with_max_time_check.groupby(["mergeID"]).fireID.unique().reset_index()
                    
    # Check when the fireID and mergeID started/stopped. 
    get_merge_start = plot_first[["fireID", "t"]].rename(columns={"fireID":"mergeID", 
                                                           "t":"mergeID_start_t"})
    get_fireID_start =  plot_first[["fireID", "t"]].rename(columns={ "t":"fireID_start_t"})

    id_map = id_with_max_time_check.merge(get_merge_start, on = ["mergeID"])
    id_map = id_map.merge(get_fireID_start, on = ["fireID"])
    id_map = id_map[["fireID","fireID_start_t",  "fireID_end_t", "mergeID", "mergeID_start_t", "mergeID_t"]]
    id_map["time_diff_fireIDend_mergeIDstart"] = id_map.fireID_end_t.astype('datetime64[ns]') - id_map.mergeID_start_t.astype('datetime64[ns]') ## Negative means that mergeID started after fireID ended
    
    # Subset to IDs where one fire "ended" before the next fire began                
    only_IDs_with_negative_dates = id_map[id_map.time_diff_fireIDend_mergeIDstart.dt.days < 0]
    
    # Go through an reindex just the IDs that overlap in space but not time                
    fires["old_id"] = ""
    fire_ids = only_IDs_with_negative_dates.mergeID.unique() ## Gives the IDs of fires to be merged into anouther fire

    for i in fire_ids:
        min_t = id_with_max_time_check[id_with_max_time_check.mergeID == i].fireID_end_t.min()
        sm_id_map = id_with_max_time_check[(id_with_max_time_check.mergeID == i) & (id_with_max_time_check.fireID_end_t == min_t)]
        if(len(sm_id_map) == 0):
            print("ID", i, " only merges with self")
            fires["old_id"][fires.fireID == i] = i
        else:
            if(len(sm_id_map) != 1):
                print("There are two perimeters that intersect with ID",i, " that started at the same time.")
                print(sm_id_map)
                break

            fires["old_id"][fires.fireID == i] = i
            fires["fireID"][fires.fireID == i] = str(*sm_id_map.fireID.values)
                    
    return(fires)

In [282]:
path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/20_days_fire_stats_only_718270-99999_minT20230301_maxT20230830_BOREAL_NRT_3571_DPSstation20231120.csv" ## looking for lightning 20 days before start
fires = prep_fire_files(path)
new_fires = remerge_largefire(fires)

/tmp/ipykernel_677/1327687407.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires["old_id"][fires_test.fireID == i] = i
/tmp/ipykernel_677/1327687407.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires["fireID"][fires_test.fireID == i] = str(*sm_id_map.fireID.values)
/tmp/ipykernel_677/1327687407.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires["old_id"][fires_test.fireID == i] = i
/tmp/ipykernel_677/1327687407.

,fireID,t,geometry,Unnamed: 0,InterCloud,lat,lon,current_mag,error_elps,num_station,...,ISI,FWI,OBSMINUTEDIFF_TEMP,OBSMINUTEDIFF_RH,OBSMINUTEDIFF_WDSPD,ISPRECREPORTED,OBSMINUTEDIFF_SNOW,OBSMINUTEDIFF_VIS,pre_fire,old_id
0,9203,2023-06-01 21:50:48.378,None,0,0.0,49.050685,-75.065585,7634.0,20.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
1,9203,2023-06-01 21:50:48.378,None,1,0.0,49.050685,-75.065585,7634.0,20.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
2,9203,2023-06-01 21:50:48.378,None,0,0.0,49.050685,-75.065585,7634.0,20.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
3,9203,2023-06-01 21:50:48.378,None,1,0.0,49.050685,-75.065585,7634.0,20.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,
4,9203,2023-06-01 22:38:58.228,None,2,0.0,49.065694,-75.045426,8725.0,16.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160107,9268,2023-07-03 07:00:00,None,1152,NaN,NaN,NaN,NaN,NaN,NaN,...,2.875352,15.279587,0.0,0.0,0.0,1.0,0.0,0.0,False,
160108,9268,2023-07-03 08:00:00,None,1153,NaN,NaN,NaN,NaN,NaN,NaN,...,3.336880,17.095431,0.0,0.0,0.0,1.0,0.0,0.0,False,
160109,9268,2023-07-03 09:00:00,None,1154,NaN,NaN,NaN,NaN,NaN,NaN,...,5.282460,23.834460,0.0,0.0,0.0,1.0,0.0,0.0,False,
160110,9268,2023-07-03 10:00:00,None,1155,NaN,NaN,NaN,NaN,NaN,NaN,...,4.814687,22.323336,0.0,0.0,0.0,1.0,0.0,0.0,False,


In [274]:
path = "/projects/old_shared/fire_weather_vis/Lightning_analysis/fwi_timeline_merge/20_days_fire_stats_only_718270-99999_minT20230301_maxT20230830_BOREAL_NRT_3571_DPSstation20231120.csv" ## looking for lightning 20 days before start

from shapely import wkt




#gpd_fires = gpd.GeoDataFrame(fires, crs = "3571", geometry  = 'geometry')


fires = pd.read_csv(path)

fires = fires.rename(columns={"geometry":"csv_geometry"})
fires.t = fires.t.astype("str")
fires.fireID  = fires.fireID.astype("str")
#fires['csv_geometry'] =fires['csv_geometry'].apply(wkt.loads)
fires_geom = gpd.read_file(path, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")
fires_geom.t = fires_geom.t.astype("str")
fires_geom.fireID  = fires_geom.fireID.astype("str")

fires = fires_geom[["fireID", "t", "geometry"]].merge(fires, on=["fireID", "t"], how = "left")

fires = fires.set_crs("3571")
fires.columns

Index(['fireID', 't', 'geometry', 'Unnamed: 0', 'InterCloud', 'lat', 'lon',
       'current_mag', 'error_elps', 'num_station', 'num_candidates',
       'num_strikes', 'num_strikes_10_days', 'no_strikes_in_time',
       'viirs_pix_count', 'n_pixels', 'n_newpixels', 'farea', 'fperim',
       'flinelen', 'duration', 'pixden', 'meanFRP', 'csv_geometry',
       'lon_centroid', 'lat_centroid', 'TEMP_C', 'RH_PERC', 'VPD_HPA',
       'WDSPD_KPH', 'PREC_MM', 'SNOWD_M', 'VIS_KM', 'FFMC', 'DMC', 'DC', 'BUI',
       'ISI', 'FWI', 'OBSMINUTEDIFF_TEMP', 'OBSMINUTEDIFF_RH',
       'OBSMINUTEDIFF_WDSPD', 'ISPRECREPORTED', 'OBSMINUTEDIFF_SNOW',
       'OBSMINUTEDIFF_VIS', 'pre_fire'],
      dtype='object')

In [28]:
first_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.min().reset_index()

last_perims = fires[~fires.geometry.isnull()].groupby("fireID").t.max().reset_index()


plot_last = fires.merge(last_perims, on = ["fireID", "t"], how = 'right')
plot_first = fires.merge(first_perims, on = ["fireID", "t"], how = 'right')

plot_last = plot_last[["fireID", "t", "geometry"]]
plot_first = plot_first[["fireID", "t", "geometry"]]


In [196]:
#plot_first

,fireID,t,geometry
0,10054,2023-06-12 12:00:00,"MULTIPOLYGON (((3942074.116 1739631.961, 39420..."
1,10140,2023-06-13 12:00:00.000,"POLYGON ((4219987.212 1362647.276, 4220260.713..."
2,10373,2023-06-16 12:00:00,"POLYGON ((4158852.818 1019497.274, 4158854.897..."
3,10406,2023-06-17 00:00:00,"POLYGON ((3898408.079 962036.358, 3897950.147 ..."
4,10435,2023-06-17 12:00:00,"POLYGON ((4400840.853 802614.719, 4400839.951 ..."
...,...,...,...
163,9612,2023-06-09 00:00:00.000,"POLYGON ((4098228.583 1286242.068, 4098227.680..."
164,9791,2023-06-11 00:00:00.000,"MULTIPOLYGON (((4172517.426 1019497.085, 41725..."
165,9837,2023-06-11 12:00:00,"POLYGON ((4015946.400 1209884.020, 4015940.156..."
166,9843,2023-06-11 12:00:00,"POLYGON ((3771628.842 1195401.453, 3771633.798..."


In [173]:
#last_perims.sjoin(last_perims, )
last_in_last = plot_last.sjoin(plot_last, how = "left", predicate = "intersects")

lil = last_in_last.groupby(["fireID_left",]).t_right.min().reset_index()
id_with_max_time = lil.merge(last_in_last[["t_right", "t_left", "geometry", "fireID_right", "fireID_left"]], on = ["t_right", "fireID_left"], how = "right")

id_with_max_time = id_with_max_time.rename(columns={"fireID_right": "mergeID", 
                                 "fireID_left" :"fireID",
                                 "t_right":"mergeID_t",
                                 "t_left":"fireID_end_t"
})

id_with_max_time_check = id_with_max_time[id_with_max_time.fireID_end_t < id_with_max_time.mergeID_t]

### FireID the earlier perimeter that later perimeters are merged into
fireID_with_merge = id_with_max_time_check.groupby(["fireID"]).mergeID.unique().reset_index() 

mergeID_with_fireID  = id_with_max_time_check.groupby(["mergeID"]).fireID.unique().reset_index()


type(fireID_with_merge)

pandas.core.frame.DataFrame

In [175]:
#last_in_last[last_in_last.fireID_left == '12149']
last_in_last

,fireID_left,t_left,geometry,index_right,fireID_right,t_right
0,10054,2023-07-04 12:00:00,"MULTIPOLYGON (((3942581.058 1741423.104, 39425...",14,10713,2023-07-16 00:00:00.000
0,10054,2023-07-04 12:00:00,"MULTIPOLYGON (((3942581.058 1741423.104, 39425...",0,10054,2023-07-04 12:00:00
0,10054,2023-07-04 12:00:00,"MULTIPOLYGON (((3942581.058 1741423.104, 39425...",104,8244,2023-06-25 12:00:00.000
1,10140,2023-07-03 12:00:00.000,"POLYGON ((4219987.212 1362647.276, 4220260.713...",1,10140,2023-07-03 12:00:00.000
2,10373,2023-06-18 00:00:00,"POLYGON ((4158858.906 1019535.029, 4158864.804...",2,10373,2023-06-18 00:00:00
...,...,...,...,...,...,...
166,9843,2023-07-28 12:00:00,"MULTIPOLYGON (((3769458.019 1204357.020, 37694...",166,9843,2023-07-28 12:00:00
166,9843,2023-07-28 12:00:00,"MULTIPOLYGON (((3769458.019 1204357.020, 37694...",25,10946,2023-06-22 00:00:00.000
166,9843,2023-07-28 12:00:00,"MULTIPOLYGON (((3769458.019 1204357.020, 37694...",21,10809,2023-06-22 00:00:00.000
167,9844,2023-06-20 12:00:00.000,"POLYGON ((3761647.815 1210495.430, 3761648.441...",167,9844,2023-06-20 12:00:00.000


In [176]:
#lil[lil.fireID_left == '12149']
lil

,fireID_left,t_right
0,10054,2023-06-25 12:00:00.000
1,10140,2023-07-03 12:00:00.000
2,10373,2023-06-18 00:00:00
3,10406,2023-06-24 00:00:00
4,10435,2023-06-22 00:00:00.000
...,...,...
163,9612,2023-07-03 12:00:00.000
164,9791,2023-06-18 00:00:00
165,9837,2023-07-03 12:00:00
166,9843,2023-06-20 12:00:00.000


In [177]:
#id_with_max_time_check[id_with_max_time_check.fireID == '10054']
id_with_max_time_check

,fireID,mergeID_t,fireID_end_t,geometry,mergeID
0,10054,2023-07-16 00:00:00.000,2023-07-04 12:00:00,"MULTIPOLYGON (((3942581.058 1741423.104, 39425...",10713
5,10373,2023-07-30 12:00:00.000,2023-06-18 00:00:00,"POLYGON ((4158858.906 1019535.029, 4158864.804...",9791
7,10406,2023-08-06 12:00:00,2023-06-24 00:00:00,"MULTIPOLYGON (((3902174.891 959544.320, 390217...",8495
17,10538,2023-07-25 12:00:00,2023-06-25 12:00:00.000,"POLYGON ((4162410.512 1108954.580, 4162409.545...",8619
18,10546,2023-07-15 00:00:00,2023-06-22 00:00:00.000,"POLYGON ((4395193.489 817951.937, 4395201.153 ...",10435
...,...,...,...,...,...
332,9346,2023-07-30 12:00:00.000,2023-06-07 00:00:00.000,"POLYGON ((4318817.350 1002150.533, 4318814.892...",8570
334,9509,2023-07-16 12:00:00.000,2023-06-13 00:00:00.000,"POLYGON ((4111961.475 1263580.645, 4111945.074...",9188
337,9612,2023-07-16 12:00:00.000,2023-07-03 12:00:00.000,"MULTIPOLYGON (((4093563.547 1284857.011, 40935...",10580
340,9837,2023-07-16 12:00:00.000,2023-07-03 12:00:00,"MULTIPOLYGON (((4020017.318 1215302.652, 40200...",10516


In [221]:
len(id_with_max_time_check[id_with_max_time_check.mergeID == '9203'])

0

In [229]:
min_t = id_with_max_time_check[id_with_max_time_check.mergeID == '8474'].fireID_end_t.min()
sm_id_map = id_with_max_time_check[(id_with_max_time_check.mergeID == '8474') & (id_with_max_time_check.fireID_end_t == min_t)]
#str(*sm_id_map.fireID.values)
sm_id_map

,fireID,mergeID_t,fireID_end_t,geometry,mergeID
307,8737,2023-07-16 00:00:00.000,2023-06-03 00:00:00,"MULTIPOLYGON (((4357980.005 1012918.097, 43579...",8474
308,8747,2023-07-16 00:00:00.000,2023-06-03 00:00:00,"POLYGON ((4354870.408 1095405.421, 4354852.651...",8474


In [179]:
#fireID_with_merge[fireID_with_merge.fireID == "10054"]
#fireID_with_merge[fireID_with_merge.fireID == "12596"]
fireID_with_merge

,fireID,mergeID
0,10054,[10713]
1,10373,[9791]
2,10406,[8495]
3,10538,[8619]
4,10546,"[10435, 8610]"
5,10789,"[8647, 8570]"
6,10809,[9843]
7,10946,[9843]
8,10976,[10749]
9,11713,[8495]


In [180]:
mergeID_with_fireID

,mergeID,fireID
0,10054,[8244]
1,10435,"[10546, 8610]"
2,10516,[9837]
3,10547,[8624]
4,10577,[8723]
5,10580,[9612]
6,10713,"[10054, 8244]"
7,10714,[8244]
8,10737,[9267]
9,10749,[10976]


In [239]:
get_merge_start = plot_first[["fireID", "t"]].rename(columns={"fireID":"mergeID", 
                                                           "t":"mergeID_start_t"})
get_fireID_start =  plot_first[["fireID", "t"]].rename(columns={ "t":"fireID_start_t"})

id_map = id_with_max_time_check.merge(get_merge_start, on = ["mergeID"])
id_map = id_map.merge(get_fireID_start, on = ["fireID"])
id_map = id_map[["fireID","fireID_start_t",  "fireID_end_t", "mergeID", "mergeID_start_t", "mergeID_t"]]

In [270]:
# fires["old_id"] = ""
# fire_ids = fires.fireID.unique()

# for i in fire_ids:
#     min_t = id_with_max_time_check[id_with_max_time_check.mergeID == i].fireID_end_t.min()
#     sm_id_map = id_with_max_time_check[(id_with_max_time_check.mergeID == i) & (id_with_max_time_check.fireID_end_t == min_t)]
#     if(len(sm_id_map) == 0):
#         print("ID", i, " only merges with self")
#         fires["old_id"][fires.fireID == i] = i
#     else:
#         if(len(sm_id_map) != 1):
#             print("There are two perimeters that intersect with ID",i, " that started at the same time.")
#             print(sm_id_map)
#             break

#         fires["old_id"][fires.fireID == i] = i
#         fires["fireID"][fires.fireID == i] = str(*sm_id_map.fireID.values)

    

In [233]:
sm_id_map.fireID.unique()

array(['8737', '8747'], dtype=object)

In [193]:
# def get_listed_ids(quebec_stats, col_name = "mergeID"):
#     newlist = [x.strip('][\n').split(' ') for x in quebec_stats[col_name].unique()]
#     newlist = list(chain(*newlist))
#     newlist = [x.replace('\n', ' ') for x in newlist]
#     newlist = unique(newlist)
#     return(newlist)

# #[*fireID_with_merge["mergeID"]]
# newlist = [x.strip('][').split(' ') for x in fireID_with_merge["mergeID"]]

In [90]:
### Go through IDs and reassign ID if it is merged into a different perimeter

In [90]:
#newlist

In [89]:
#lil.groupby(["fireID_left"]).filter(lambda x: x['t_right'] ==  x['t_right'].max())

In [91]:
#plot_first.fireID.unique()

In [92]:
#str(plot_first.t[0:2].values)

In [242]:
last_in_last[last_in_last.fireID_right == '8474'].explore()

In [200]:
plot_first[plot_first.fireID == "10713"].explore()

In [237]:
gdf_id_map = gpd.GeoDataFrame(sm_id_map)

In [238]:
gdf_id_map.explore()

In [241]:
id_map[id_map.mergeID == '8474']

,fireID,fireID_start_t,fireID_end_t,mergeID,mergeID_start_t,mergeID_t
59,8538,2023-06-01 12:00:00,2023-06-23 00:00:00,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
60,8562,2023-06-02 00:00:00,2023-06-05 12:00:00,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
61,8563,2023-06-02 00:00:00.000,2023-06-22 12:00:00.000,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
63,8620,2023-06-02 12:00:00.000,2023-06-06 00:00:00.000,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
64,8632,2023-06-02 12:00:00.000,2023-06-12 12:00:00.000,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
65,8677,2023-06-02 12:00:00.000,2023-06-20 00:00:00.000,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
66,8679,2023-06-02 12:00:00.000,2023-06-05 12:00:00.000,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
68,8720,2023-06-03 00:00:00.000,2023-06-06 12:00:00.000,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
70,8737,2023-06-03 00:00:00,2023-06-03 00:00:00,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000
72,8747,2023-06-03 00:00:00,2023-06-03 00:00:00,8474,2023-06-01 00:00:00.000,2023-07-16 00:00:00.000


In [245]:
id_map[id_map.mergeID == '12602']

,fireID,fireID_start_t,fireID_end_t,mergeID,mergeID_start_t,mergeID_t
46,12642,2023-07-07 12:00:00.000,2023-07-11 12:00:00.000,12602,2023-07-07 00:00:00.000,2023-07-15 12:00:00.000


In [244]:
plot_first.explore()

In [248]:
id_map["time_diff_fireIDend_mergeIDstart"] = id_map.fireID_end_t.astype('datetime64[ns]') - id_map.mergeID_start_t.astype('datetime64[ns]') ## Negative means that mergeID started after fireID ended
only_IDs_with_negative_dates = id_map[id_map.time_diff_fireIDend_mergeIDstart.dt.days < 0]
only_IDs_with_negative_dates


In [279]:
only_IDs_with_negative_dates = id_map[id_map.time_diff_fireIDend_mergeIDstart.dt.days < 0]
only_IDs_with_negative_dates

,fireID,fireID_start_t,fireID_end_t,mergeID,mergeID_start_t,mergeID_t,time_diff_fireIDend_mergeIDstart
31,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,16417,2023-08-05 00:00:00,2023-08-06 12:00:00,-20 days +00:00:00
32,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,16062,2023-08-02 12:00:00.000,2023-08-06 12:00:00.000,-18 days +12:00:00
33,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,16459,2023-08-05 12:00:00.000,2023-08-06 12:00:00.000,-21 days +12:00:00
34,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,16112,2023-08-02 12:00:00.000,2023-08-06 12:00:00.000,-18 days +12:00:00
35,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,15929,2023-08-01 12:00:00.000,2023-08-06 12:00:00.000,-17 days +12:00:00
36,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,16520,2023-08-05 12:00:00,2023-08-06 12:00:00,-21 days +12:00:00
37,12596,2023-07-07 00:00:00.000,2023-07-16 00:00:00.000,16519,2023-08-05 12:00:00,2023-08-06 12:00:00,-21 days +12:00:00
38,12603,2023-07-07 00:00:00.000,2023-07-15 00:00:00.000,16062,2023-08-02 12:00:00.000,2023-08-06 12:00:00.000,-19 days +12:00:00
39,12603,2023-07-07 00:00:00.000,2023-07-15 00:00:00.000,16459,2023-08-05 12:00:00.000,2023-08-06 12:00:00.000,-22 days +12:00:00
40,12603,2023-07-07 00:00:00.000,2023-07-15 00:00:00.000,16112,2023-08-02 12:00:00.000,2023-08-06 12:00:00.000,-19 days +12:00:00


In [259]:
#id_map.time_diff_fireIDend_mergeIDstart.dt.days < 0 

In [268]:
last_in_last[last_in_last.fireID_right == '13906'].explore()

In [275]:
fires_test = fires

In [276]:
fires_test["old_id"] = ""
fire_ids = only_IDs_with_negative_dates.mergeID.unique() ## Gives the IDs of fires to be merged into anouther fire

for i in fire_ids:
    min_t = id_with_max_time_check[id_with_max_time_check.mergeID == i].fireID_end_t.min()
    sm_id_map = id_with_max_time_check[(id_with_max_time_check.mergeID == i) & (id_with_max_time_check.fireID_end_t == min_t)]
    if(len(sm_id_map) == 0):
        print("ID", i, " only merges with self")
        fires_test["old_id"][fires_test.fireID == i] = i
    else:
        if(len(sm_id_map) != 1):
            print("There are two perimeters that intersect with ID",i, " that started at the same time.")
            print(sm_id_map)
            break

        fires_test["old_id"][fires_test.fireID == i] = i
        fires_test["fireID"][fires_test.fireID == i] = str(*sm_id_map.fireID.values)

/tmp/ipykernel_677/1709106897.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires_test["old_id"][fires_test.fireID == i] = i
/tmp/ipykernel_677/1709106897.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires_test["fireID"][fires_test.fireID == i] = str(*sm_id_map.fireID.values)
/tmp/ipykernel_677/1709106897.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires_test["old_id"][fires_test.fireID == i] = i
/tmp/ipykernel_

In [278]:
fires_test.old_id.unique()

array(['', '16417', '15929', '13906', '16519', '16459', '13124', '16062',
       '16520', '16112'], dtype=object)